# Aula 29 – Arquiteturas de Redes Neurais Convolucionais

## 1. Introdução

Como discutido na Aula 26, existem diferentes arquiteturas de RNC que tentam otimizar o processo de classificação. Esta aula tem o objetivo de citar algumas das principais e comentar as ideias por trás delas, sem a intenção de aprofundamento, pois o assunto é muito extenso e complexo, cabendo inclusive uma disciplina específica para isso.

Como já falado, as RNC não são criação recente, tendo a ideia geral de utilizar neurônios como filtros para extrair características de imagens sido colocada ainda nos anos 80. Entretanto, a disponibilidade de poder de processamento e armazenamento de dados permitiu um grande crescimento da área nos anos 2010, identificando-se aí o que é chamado de Deep Learning, pois tratam de redes com muitas camadas (profundas).

Até 2011, competição ImageNet Large Scale Visual Recognition Challenge (ILSVRC) tinha como vencedores as chamadas redes neurais rasas (ImageNet é um conjunto de dados marcados e classificados para diversos objetos). A partir de 2012 começaram a ser introduzidas redes que podiam ser chamadas de profundas e a performance em relação a 2011 da rede vencedora melhorou em 37% (sendo que 2011 e 2010 a performance foi praticamente a mesma).

A vencedora em 2011 foi a AlexNet, basicamente trazia a estrutura de RNC já discutida na aula 26, mas com 8 camadas que, apesar de parecer pouco, já é bastante pesado computacionalmente e só obteve sucesso graças à implementação usando GPU. Melhoras em sua implementação permitiram um resultado melhorado e novamente ganhou o desafio de 2013 (essa implementação é conhecida como ZFNet e tem melhores ajustes de hiperparâmetros, como quantidade, tamanho e deslocamento dos filtros).

Em 2014 a arquitetura vencedora em localização foi a VGG (mas não em classificação), com 19 camadas e performance 38% melhor em relação à ZFNet. A VGG possuía filtros ainda menores (convoluções 3x3) com deslocamento unitário. Tais filtros menores exigiam mais camadas. Uma camada de convolução 7x7 corresponde a três camadas de convolução 3x3.

A vencedora em classificação em 2014 foi a GoogLeNet, com 22 camadas, com melhora de 8% em relação à VGG e 43% em relação à ZFNet. A GoogLeNet trouxe de novidade a criação de módulos, chamados “Inception”, que aplica filtros em paralelo e depois concatena o resultado. Isso aumentou consideravelmente a complexidade de cada camada, mas também melhorou a eficiência da extração de características. Para ajudar no treinamento, classificações são feitas nas camadas intermediárias para melhorar a eficiência do ajuste de parâmetros, pois a correção baseada apenas na saída final da rede seria ineficiente com tantas camadas e de alta complexidade.

Em 2015 houve uma “revolução de profundidade” com a vencedora ResNet. Com 152 camadas, conseguiu um resultado ainda 47% melhor em relação à já excelente GoogLeNet. Para trabalhar com tantas camadas, a rede introduz a ideia de “conexões residuais”. Treinar uma rede muito profunda é um problema de otimização pesado, pois o número de parâmetros a ajustar cresce muito. A ResNet trabalha com a hipótese de que treinar parâmetros (camadas) de “resíduo” da imagem é mais eficiente que treinar para a imagem original. Então “bloco residual” para frente exatamente seus valores de entrada e, na saída, soma-se a entrada que “saltou” à saída filtrada. Ou seja, o ajuste que deve ser feito nos filtros é apenas para adequar a diferença entre a saída e a entrada do bloco. Isso de fato se mostrou mais eficiente no treinamento, permitindo aumentar o número de camadas e, consequentemente, a performance de detecção da rede.

Em 2017 (último campeonato realizado), a vencedora aplicou a ideia da ResNet para treinar uma rede com blocos residuais formados por camadas de módulos Inceptions, resultando na Inception-v4 (outras vieram ainda antes).

## 2. Material Complementar

http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture9.pdf

https://www.youtube.com/watch?v=DAOcjicFr1Y

http://www.image-net.org/

## 3. Exercícios

1. Keras é uma API Python para Deep Learning. Construir redes neurais convolucionais utilizando essa API é basicamente um processo de empilhar camadas através de chamadas de funções. Realize o seguinte tutorial para construir uma rede simples para classificação de caracteres: https://keras.io/examples/vision/mnist_convnet/



In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [5]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/15
422/422 [==============================] - 90s 212ms/step - loss: 0.3667 - accuracy: 0.8897 - val_loss: 0.0828 - val_accuracy: 0.9785
Epoch 2/15
422/422 [==============================] - 88s 208ms/step - loss: 0.1128 - accuracy: 0.9664 - val_loss: 0.0612 - val_accuracy: 0.9832
Epoch 3/15
422/422 [==============================] - 88s 208ms/step - loss: 0.0827 - accuracy: 0.9745 - val_loss: 0.0445 - val_accuracy: 0.9878
Epoch 4/15
422/422 [==============================] - 88s 209ms/step - loss: 0.0709 - accuracy: 0.9787 - val_loss: 0.0455 - val_accuracy: 0.9868
Epoch 5/15
422/422 [==============================] - 89s 210ms/step - loss: 0.0621 - accuracy: 0.9803 - val_loss: 0.0389 - val_accuracy: 0.9900
Epoch 6/15
422/422 [==============================] - 89s 210ms/step - loss: 0.0553 - accuracy: 0.9830 - val_loss: 0.0344 - val_accuracy: 0.9910
Epoch 7/15
422/422 [==============================] - 88s 208ms/step - loss: 0.0497 - accuracy: 0.9842 - val_loss: 0.0332 - val_ac

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.025909854099154472
Test accuracy: 0.9919000267982483
